In [ ]:
import sys
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats
#sys.path






import datetime
import math
import time
import itertools


from pymongo import MongoClient



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [ ]:

class MongoConnection(object):
    def __init__(self, cxnSettings, **kwargs):
        self.settings = cxnSettings
        self.mongoURI = self._constructURI()
        self.connect(**kwargs)
        self.ensure_index()

    def _constructURI(self):
        '''
        Construct the mongo URI
        '''
        mongoURI = 'mongodb://'
        #User/password handling
        if 'user'in self.settings and 'password' in self.settings:
            mongoURI += self.settings['user'] + ':' + self.settings['password']
            mongoURI += '@'
        elif 'user' in self.settings:
            print('Missing password for given user, proceeding without either')
        elif 'password' in self.settings:
            print('Missing user for given passord, proceeding without either')
        #Host and port
        try:
            mongoURI += self.settings['host'] + ':'
        except KeyError:
            print('Missing the hostname. Cannot connect without host')
            sys.exit()
        try:
            mongoURI += str(self.settings['port'])
        except KeyError:
            print('Missing the port. Substituting default port of 27017')
            mongoURI += str('27017')
        return mongoURI

    def connect(self, **kwargs):
        '''
        Establish the connection, database, and collection
        '''
        self.connection = MongoClient(self.mongoURI, **kwargs)
        #########
        try:
            self.db = self.connection[self.settings['db']]
        except KeyError:
            print("Must specify a database as a 'db' key in the settings file")
            sys.exit()
        #########
        try:
            self.collection = self.db[self.settings['collection']]
        except KeyError:
            print('Should have a collection.', end='')
            print('Starting a collection in database', end='')
            print(' for current connection as test.')
            self.collection = self.db['test']

    def tearDown(self):
        '''
        Closes the connection
        '''
        self.connection.close()

    def ensure_index(self):
        '''
        Ensures the connection has all given indexes.
        indexes: list of (`key`, `direction`) pairs.
            See docs.mongodb.org/manual/core/indexes/ for possible `direction`
            values.
        '''
        if 'indexes' in self.settings:
            for index in self.settings['indexes']:
                self.collection.ensure_index(index[0], **index[1])



#############################

merged_papers_settings = {
    "host": "chicago.chem-eng.northwestern.edu",
    "port": "27017",
    "db": "web_of_science_aux",
    "collection": "merged_papers",
    "user": "mongoreader",
    "password": "emptycoffeecup"
}

papers_con = MongoConnection(merged_papers_settings)




#############################

dais_settings = {
    "host": "chicago.chem-eng.northwestern.edu",
    "port": "27017",
    "db": "web_of_science_aux",
    "collection": "ut_dais_all",
    "user": "mongoreader",
    "password": "emptycoffeecup"
}

dais_con = MongoConnection(dais_settings)



In [ ]:
cursor = papers_con.collection.find({"UT":"000429763700010"})  
#for s in cursor:
    #print(s)

In [ ]:
# cursor = papers_con.collection.find().sort([("issue.PY",-1)]).limit(10) # i look for the 10 youngest papers (write +1 instead of -1 for the oldest)
# # for s in cursor:
# #     print(s)
# #     print('--------------')

In [ ]:
                                          # df_reference_cite_plos_merged     (original with ALL columns and all entries, without removing any references)

#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl', 'rb'))

#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))
%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_with_team_expertise_and_recycled_ref.pkl', 'rb'))





print ("done loading pickles", df_merged.shape)


#%time plos_df = pickle.load(open('../data/plos_paper_dataframe_more_columns.pkl', 'rb'))

#%time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES.pkl', 'rb'))
%time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'rb'))



#%time plos_df = pickle.load(open('../data/plos_paper_dataframe.pkl', 'rb'))
print ("done loading pickles", plos_df.shape)
plos_df.head()




#### OJOOOO!new!! if i load the original df, i need to remove duplicate references used more than once per section and per plos paper:  (numbers go down from 8.9M to 6.9M)
# print (df_merged.shape)  
# df_merged=df_merged.drop_duplicates(subset=['paper_UT','reference_UT','reference_rank','regex_sect_index','cite_count','ref_pub_year','paper_cite_count','total_refs','plos_pub_year','plos_field'])
# print (df_merged.shape)


print (sorted(df_merged.columns))

df_merged.head()

In [ ]:
plos_df.head()

In [ ]:
#sorted(df_merged.columns)
sorted(plos_df.columns)


In [ ]:
# df_plosone = df_merged[df_merged['plos_j1']=="PLOS ONE"]
# list_plos_one = list(df_plosone.paper_UT.unique())
# len(list_plos_one)




# done loading pickles (6924269, 23)
# ['cite_count',
#  'diff_year_plos_ref',
#  'isolated_citation',
#  'log10_num_cit_paper',
#  'log10_num_cit_ref',
#  'log2_num_cit_paper',
#  'log2_num_cit_ref',
#  'occurence',
#  'paper_UT',
#  'paper_cite_count',
#  'plos_field',
#  'plos_j1',
#  'plos_pub_year',
#  'ref_field',
#  'ref_j1',
#  'ref_pub_year',
#  'reference_UT',
#  'reference_rank',
#  'regex_sect_index',
#  'rel_loc_in_sect',
#  'sect_char_pos',
#  'sect_char_total',
#  'total_refs']

In [ ]:

df_merged.head()

In [ ]:
list_plos = list(df_merged.paper_UT.unique())
len(list_plos)

In [ ]:
################  i add the plos one category (coded as numbers) to those rows corresponding to plos one paper_UTs


try:
    
    dict_UT_plos_categ = pickle.load(open('../data/dict_UT_plos_categ.pkl', 'rb'))
    print ("loaded pickle dict", len(dict_UT_plos_categ))

except:
    
    cont_found_categ=0
    start=0
    stop=1000
    dict_UT_plos_categ={}
    while stop <=159000:


        lista=list_plos[start:stop]
        cursor = papers_con.collection.find({"UT":{"$in":lista}},{"UT":1,"plos_categories":1}, no_cursor_timeout=True)  


        print (start, stop)

        for item in cursor:  # query (cursor) is an iterator (once i iterate over it once, it is empty), and every item is a dict

            UT=item["UT"]
            try:
                list_subjects=item['plos_categories']
                cont_found_categ +=1
            except KeyError:
                list_subjects =[]
            dict_UT_plos_categ[UT]=list_subjects

        stop +=1000
        start +=1000


        cursor.close()  # because i am using the no_cursor_timeout=True, i need also this, or cursor keeps waiting so ur resources are used up

    print ("done, num. UTs with category info:", cont_found_categ, "   tot UTs", len(dict_UT_plos_categ))


    with open('../data/dict_UT_plos_categ.pkl', 'wb') as handle:
         pickle.dump(dict_UT_plos_categ, handle, protocol = 2)
    print ("written:",'../data/dict_UT_plos_categ.pkl')   


    
##########################    
    
    
    
lista_values = dict_UT_plos_categ.values()
lista_diff_categ=[]
for item in lista_values:
    lista_diff_categ += item

lista_diff_categ = sorted(list(set(lista_diff_categ)))
print (len(lista_diff_categ), lista_diff_categ)
    
    
    
    
code = 0
dict_categ_code={}
for categ in lista_diff_categ:
    dict_categ_code[categ] = code
    code +=1    
print (dict_categ_code)
with open('../data/dict_categ_code.pkl', 'wb') as handle:
     pickle.dump(dict_categ_code, handle, protocol = 2)
print ("written:",'../data/dict_categ_code.pkl')   




new_dict_UT_plos_categ_codes={}
for UT in dict_UT_plos_categ:
    new_combined_code=''
    for categ in dict_UT_plos_categ[UT]:
        code = dict_categ_code[categ]
        new_combined_code += (str(code) +" ")
        
    new_dict_UT_plos_categ_codes[UT] = new_combined_code.strip()  # i just remove the last space
    
print (len(new_dict_UT_plos_categ_codes))



###################### I convert the info UT-categ into a dataframe for merging with the df_merged one:

lista_UTs=[]
lista_categ_codes=[]
for UT in new_dict_UT_plos_categ_codes:
    lista_UTs.append(UT)
    lista_categ_codes.append(new_dict_UT_plos_categ_codes[UT])
   
data_tuples = list(zip(lista_UTs,lista_categ_codes))
df_categ = pd.DataFrame(data_tuples, columns=['paper_UT','categ_codes'])

path = '../data/df_UT_plosone_categ_codes.pkl'
%time df_categ.to_pickle(path, compression='infer', protocol=2)
print ("written little df with plos")


##########################

df_merged = pd.merge(df_merged, df_categ, on='paper_UT', how='left')
df_merged.head(100)



In [ ]:

path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
%time df_merged.to_pickle(path, compression='infer', protocol=2)

print ("written:",path)  


In [ ]:
dict_categ_code

In [ ]:
sorted(df_merged.columns)

In [ ]:
def fix_paper_UT(row):
    paper_UT = row.paper_UT
    return '000'+str(paper_UT)

#df_testing = df_merged.head(1000)

df_merged['new_paper_UT'] = df_merged.apply (lambda row: fix_paper_UT(row),axis=1)
df_merged.drop(['paper_UT'], axis=1, inplace=True)
df_merged.rename(columns={'new_paper_UT': 'paper_UT'}, inplace=True)

df_merged[['paper_UT']]

In [ ]:
df_merged[['paper_UT']].head()

In [ ]:
# df_testing=df_merged.copy(deep=True)
# print (df_merged.shape, df_testing.shape)


# ###### if i upload the original merged df, then i need to select some columns, or it will be too long
# print (sorted(df_merged.columns)) 
# df_merged = df_merged[['occurence','paper_UT','reference_UT','reference_rank','regex_sect_index','cite_count','ref_pub_year',\
#                                   'paper_cite_count','total_refs','plos_pub_year' , 'sect_char_pos', 'sect_char_total' , 'plos_field','plos_j1','ref_field', 'ref_j1', ]]
# df_merged.head()

In [ ]:

lista_all_plos_UTs = pickle.load(open('../data/lista_all_plos_UTs.pkl', 'rb'))
new_lista_all_plos_UTs = ['000'+str(item) for item in lista_all_plos_UTs]   # i need to add the 000 so it matches the db UTs

print (len(lista_all_plos_UTs))

lista_all_reference_UTs = pickle.load(open('../data/lista_all_reference_UTs.pkl', 'rb'))
print (len(lista_all_reference_UTs))






lista_UTs_plos_and_ref = new_lista_all_plos_UTs+ lista_all_reference_UTs
lista_UTs_plos_and_ref =  list(set(lista_UTs_plos_and_ref))
len(lista_UTs_plos_and_ref)   #2706287 without UT repetitions



In [ ]:
sorted(df_merged.columns)

In [ ]:
def get_log_num_cit_of_ref(row):
    
    try:
        log_num_cit=np.log10(float(row.cite_count))
    except:
        print (row.plos_pub_year)
        log_num_cit_ref=0.
    return log_num_cit

###################

def get_log_num_cit_of_paper(row):
    
    try:
        log_num_cit=np.log10(float(row.paper_cite_count))
    except:
        print (row.plos_pub_year)
        log_num_cit=0.
    return log_num_cit

###################


%time df_merged['log10_num_cit_ref'] = df_merged.apply (lambda row: get_log_num_cit_of_ref(row),axis=1)
%time df_merged['log10_num_cit_paper'] = df_merged.apply (lambda row: get_log_num_cit_of_paper(row),axis=1)

#df_merged.rel_loc_in_sect.min()
    

In [ ]:
def get_log2_num_cit_of_ref(row):
    
    try:
        log_num_cit=np.log2(float(row.cite_count))
    except:
        print (row.plos_pub_year)
        log_num_cit_ref=0.
    return log_num_cit

###################

def get_log2_num_cit_of_paper(row):
    
    try:
        log_num_cit=np.log2(float(row.paper_cite_count))
    except:
        print (row.plos_pub_year)
        log_num_cit=0.
    return log_num_cit

###################


%time df_merged['log2_num_cit_ref'] = df_merged.apply (lambda row: get_log2_num_cit_of_ref(row),axis=1)
%time df_merged['log2_num_cit_paper'] = df_merged.apply (lambda row: get_log2_num_cit_of_paper(row),axis=1)

#df_merged.rel_loc_in_sect.min()
    

In [ ]:
def get_diff_publ_plos_publ_ref(row):
    
    try:
        delta_publ_year=float(row.plos_pub_year)- float(row.ref_pub_year) 
    except:
        delta_publ_year=np.nan
    return delta_publ_year

###################


%time df_merged['diff_year_plos_ref'] = df_merged.apply (lambda row: get_diff_publ_plos_publ_ref(row),axis=1)

#df_merged.rel_loc_in_sect.min()
    

In [ ]:
def get_relative_position_within_section(row):
    
    try:
        rel_pos=float(row.sect_char_pos) / float(row.sect_char_total) 
    except:
        rel_pos=np.nan
    return rel_pos 

###################


%time df_merged['rel_loc_in_sect'] = df_merged.apply (lambda row: get_relative_position_within_section(row),axis=1)

#df_merged.rel_loc_in_sect.min()

df_merged.drop(['sect_char_pos','sect_char_total','total_refs'], axis=1, inplace=True)

In [ ]:

def get_section(row):
    
    sect_num = row.regex_sect_index
    section_name = "NA"
    
    if sect_num == 0:
        section_name = "0:Intro"
    elif sect_num == 1:
        section_name = "1:Methods"
    elif sect_num == 2:
        section_name = "2:Results"
    elif sect_num == 3:
        section_name = "3:Discussion"
    elif sect_num == 4:
        section_name = "4:Results/Discussion"
    elif sect_num == 5:
        section_name = "5:Conclusion"
    elif sect_num == 6:
        section_name = "6:Mx"
    
    
    
    return section_name 

###################


%time df_merged['section'] = df_merged.apply (lambda row: get_section(row),axis=1)


print ("done. writing pickle.....")


# path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
# %time df_merged.to_pickle(path, compression='infer', protocol=2)

# print ("written:",path)  


In [ ]:



lista_options_fields= ['D RO MULTIDISCIPLINARY SCIENCES','D RO MULTIDISCIPLINARY SCIENCES','D PY MEDICINE, GENERAL & INTERNAL','D TI PARASITOLOGY', 'D YU TROPICAL MEDICINE',\
'D CU BIOLOGY', 'D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY','D CU BIOLOGY', 'D PY MEDICINE, GENERAL & INTERNAL','D QU MICROBIOLOGY', 'D TI PARASITOLOGY', 'D ZE VIROLOGY',\
'D CU BIOLOGY', 'D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY','D KM GENETICS & HEREDITY','D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY',\
'D TI PARASITOLOGY', 'D ZE VIROLOGY', 'D QU MICROBIOLOGY','D KM GENETICS & HEREDITY','D CO BIOCHEMICAL RESEARCH METHODS', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS',\
'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY','D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']


lista_options_fields= list(set(lista_options_fields))
###################

def get_new_plos_field(row, lista_options_fields):
    
    new_field=[]
    old_field=str(row.plos_field)
    #print (old_field,)
    for field in lista_options_fields:
        if field in old_field:
            new_field.append(field)
    
    return str(new_field)

###################


%time df_merged['new_plos_field'] = df_merged.apply (lambda row: get_new_plos_field(row, lista_options_fields),axis=1)

#df_merged.rel_loc_in_sect.min()



df_merged.drop(['plos_field'], axis=1, inplace=True)
df_merged.rename(columns={'new_plos_field': 'plos_field'}, inplace=True)

df_merged

In [ ]:
# df_merged = df_merged[['occurence','paper_UT','reference_UT','reference_rank','regex_sect_index','cite_count','ref_pub_year',\
#                                   'paper_cite_count','total_refs','plos_pub_year' , 'sect_char_pos', 'sect_char_total' ,'plos_field']]


df_merged[['occurence', 'paper_UT', 'reference_UT','regex_sect_index','sect_char_pos', 'isolated_citation']].sort_values(by=['paper_UT','regex_sect_index','sect_char_pos']).head(10000)


In [ ]:
def isolated_or_group_citation(row):
    
    window=4  # max distance (in units of characters) to determine if a reference is cluster with anothers or not
    
    paper_UT = row.paper_UT
    reference_UT = row.reference_UT
    regex_sect_index = row.regex_sect_index
    sect_char_pos = row.sect_char_pos
    #print ("\n",paper_UT,reference_UT,regex_sect_index,sect_char_pos)
    
    df_selection=df_merged[ (df_merged['paper_UT'] == paper_UT) &  (df_merged['regex_sect_index'] == regex_sect_index) ]   
    
    #print (df_selection.shape)
    list_positions=sorted(list(df_selection.sect_char_pos))
    
    #print (list_positions)
    lista_diff_pos= sorted([abs(item - sect_char_pos ) for item in list_positions])[1:]
    #print (lista_diff_pos)
     
    if len(lista_diff_pos)>0:
        if min(lista_diff_pos) > window:
            #print (min(lista_diff_pos), paper_UT,reference_UT)#"isolated ref!")
            return 1   # isolated reference
        else:
            return 0
    else:
        return 0
    


###################

#df_testing=df_merged.head(10000)

%time df_merged['isolated_citation'] = df_merged.apply (lambda row: isolated_or_group_citation(row),axis=1)
df_merged

In [ ]:

def get_new_paper_UT(row):
    
    new_UT= '000'+str(row.paper_UT)
   
    return new_UT

###################


%time df_merged['new_paper_UT'] = df_merged.apply (lambda row: get_new_paper_UT(row),axis=1)

df_merged.drop(['paper_UT'], axis=1, inplace=True)
df_merged.rename(columns={'new_paper_UT': 'paper_UT'}, inplace=True)

In [ ]:

path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
%time df_merged.to_pickle(path, compression='infer', protocol=2)

print ("written:",path)  


In [ ]:
df_merged.head(100)

In [ ]:
sorted(df_merged.columns)

In [ ]:


# path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns_.pkl'
# %time df_merged.to_pickle(path, compression='infer', protocol=2)

# print ("written:",path)  



In [ ]:
df_merged.head()

In [ ]:
######## to re-add the number_of_references field that got dropped:

%time df_merged_OLD = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_cols.pkl', 'rb'))
print ("done loading pickles", df_merged_OLD.shape)


df_plos_num_ref = df_merged_OLD[['paper_UT', 'total_refs']].drop_duplicates()


## ojo! the old df
df_plos_num_ref['new_paper_UT'] = df_plos_num_ref.apply (lambda row: fix_paper_UT(row),axis=1)
df_plos_num_ref.drop(['paper_UT'], axis=1, inplace=True)
df_plos_num_ref.rename(columns={'new_paper_UT': 'paper_UT'}, inplace=True)






df_merged = pd.merge(df_merged, df_plos_num_ref, on='paper_UT', how='left')
df_merged.head()


df_merged.drop(['total_refs_x'], axis=1, inplace=True)
df_merged.rename(columns={'total_refs_y': 'total_refs'}, inplace=True)


path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
%time df_merged.to_pickle(path, compression='infer', protocol=2)
print ("written little df with plos")

In [ ]:

sorted(df_merged.columns)

In [ ]:
df_merged[['paper_UT','total_refs']].head()

In [ ]:
##############  for classifying self-citations non self-citations:
#####  i read the gzip files with the disambig author info from WOS, and filter out only the ones i care about (those papers included in the PLOS db):



try:       
                                 
    %time df_disamb = pickle.load(open('../data/df_disambig_filtered.pkl', 'rb'))
    print ("done loading pickles", df_disamb.shape)
    df_disamb.head()


except:


    path_disamb='/home/workspace/scibio/resources/rbusa/rbusa_main_v1_1/disambiguation/wos_dais/'

    lista_files=[]

    for i in range(320):
        i += 1
        file_name=path_disamb+'wos_dais_all_batch'+str(i)+'.csv.gz'    
        lista_files.append(file_name)
    ##############





    cont_tot=0
    cont_filtered=0
    ########### concatenate a list of (gzip) files into one single pandas dataframe:
    frame = pd.DataFrame()
    list_ = []
    for file_ in lista_files:
        print (file_)
        df = pd.read_csv(file_, compression = 'gzip', dtype=object, index_col=None, header=0)   # OJO!!!!!!!!!! i need to force it to read it as object (that is, as strings) so it doesnt remove the 000 at the beginning of some UTS!!!
        cont_tot += len(df)
        df = df[df['WOS'].isin(lista_UTs_plos_and_ref)]  # i directly filter out what i dont need, to save space
        cont_filtered += len(df)
        list_.append(df)

    df_disamb = pd.concat(list_)
    print (cont_tot, cont_filtered)    # 208,042,832         14,382,344


    df_disamb.rename(columns={'WOS': 'UT'}, inplace=True)
    print (df_disamb.shape)     # (14382344, 3)





    path = '../data/df_disambig_filtered.pkl'
    %time df_disamb.to_pickle(path, compression='infer', protocol=2)

    print ("written:",path)  
    df_disamb.head()
    
    
    
    
#####################


###### i build the dict UT-list_authors that i will need for classifying citations as self or not self citations:
try:
    
    %time dict_UT_list_authors = pickle.load(open('../data/dict_UT_list_authors.pkl', 'rb'))
    print ("done loading dict pickle", len(dict_UT_list_authors))

except:

    dict_UT_list_authors={}    

    for item in df_disamb.groupby(['UT']):  ### OJO!!!! IF I TRY TO DO IT WITH : for DAIS in df_disamb.DAIS.unique():      df_select= df_disamb [df_disamb ['DAIS']== DAIS]  IT would TAKE A MONTH TO RUN!!!
        #print (item, "  ---",len(item[1].UT))
        UT=item[0]
        dict_UT_list_authors[UT]=list(item[1].DAIS)


    print ("done. now writing pickle.....")

    with open('../data/dict_UT_list_authors.pkl', 'wb') as handle:
         pickle.dump(dict_UT_list_authors, handle, protocol = 2)
    print ("written:",'../data/dict_UT_list_authors.pkl')   


    
    
##############



def get_self_citation(row, dict_UT_list_authors, cont_missing_records):
    
    paper_UT = row.paper_UT
    ref_UT = row.reference_UT
    
    self_citation=0
    try:
        lista_DAIS_paper = dict_UT_list_authors[paper_UT]
    except KeyError:
        lista_DAIS_paper = []
        cont_missing_records +=1
        
        
    try:
        lista_DAIS_ref = dict_UT_list_authors[ref_UT]
    except KeyError:
        lista_DAIS_ref = []
        cont_missing_records +=1
        
#     print (paper_UT,lista_DAIS_paper ," -----",ref_UT, lista_DAIS_ref, set(lista_DAIS_paper) & set(lista_DAIS_ref))
#     input()
    
    if len(set(lista_DAIS_paper) & set(lista_DAIS_ref)) >0:  # if the citing paper and the reference paper share at least one author
        self_citation = 1
       # print ("self-cit:",paper_UT," --> ", ref_UT)
        
        
        
    
    return self_citation

###################

cont_missing_records=0
%time df_merged['self_citation'] = df_merged.apply (lambda row: get_self_citation(row, dict_UT_list_authors, cont_missing_records), axis=1)

print ("done. missing records (plos and/or ref):", cont_missing_records, "\nwriting pickle.....")





In [ ]:
############## print out the new dataframe with extra columns:
#################################
df_merged.drop([ 'log10_num_cit_paper', 'log10_num_cit_ref' ], axis=1, inplace=True)


path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
%time df_merged.to_pickle(path, compression='infer', protocol=2)

print ("written:",path)  
sorted(df_merged.columns)


In [ ]:
# ##### adding number of team members in a paper

# def get_number_authors(row):
     
#     paper_UT = row.paper_UT
       
  
#     #### i get the list of disambiguated author ID for the paper
#     result_query = papers_con.collection.find_one({"UT":paper_UT},{"AU":1})  
#    # print (result_query) ##  example: {'_id': ObjectId('54d3bdcdec29bd464368e4a8'), 'AU': [{'AU': 'Loffler, S', 'DAIS': 19993061}, {'AU': 'Jessen, J', 'DAIS': 37335135}, {'AU': 'Schmid, T', 'DAIS': 30805206}, {'AU': 'Porksen, U', 'DAIS': 63190680}]}

#     list_dict_authors = result_query['AU']  # the result of find_one is a dictionary, NOT an iterator!  And the, result_query['AU'] is a list with all the authors of the UT paper        
#     number_authors = len(list_dict_authors) 
        
    
#     return number_authors

# ###################

# %time plos_df['number_authors'] = plos_df.apply (lambda row: get_number_authors(row), axis=1)

# ######################


# path = '../data/plos_paper_dataframe_more_columns.pkl'
# %time plos_df.to_pickle(path, compression='infer', protocol=2)
# print ("written little df with plos")

In [ ]:

###### i get a dictionary for paper_UT  - publication year:

plos_simple = plos_df[['paper_UT','plos_pub_year']]
dict_aux_UT_year = dict(zip(plos_simple.paper_UT, plos_simple.plos_pub_year))
for UT in dict_aux_UT_year:
    dict_aux_UT_year[UT] = int(dict_aux_UT_year[UT])
dict_aux_UT_year

In [ ]:
    ###   DONE!   i add the number of authors in each plos paper


    try:
        dict_UT_paper_number_authors = pickle.load(open('../data/dict_UT_paper_number_authors.pkl', 'rb'))    

    except:    


        dict_UT_paper_number_authors = {}
        cont = 1
        for paper_UT in list_paper_UT:



            ######### i get the number of team members for each plos paper UT:

            result_query = papers_con.collection.find_one({"UT":paper_UT},{"AU":1})  
           # print (result_query) ##  example: {'_id': ObjectId('54d3bdcdec29bd464368e4a8'), 'AU': [{'AU': 'Loffler, S', 'DAIS': 19993061}, {'AU': 'Jessen, J', 'DAIS': 37335135}, {'AU': 'Schmid, T', 'DAIS': 30805206}, {'AU': 'Porksen, U', 'DAIS': 63190680}]}

            list_dict_authors = result_query['AU']  # the result of find_one is a dictionary, NOT an iterator!  And the, result_query['AU'] is a list with all the authors of the UT paper        
            number_authors = len(list_dict_authors) 
            dict_UT_paper_number_authors[paper_UT] = number_authors#get_number_authors(paper_UT)

            print (cont)
            cont += 1



        with open('../data/dict_UT_paper_number_authors.pkl', 'wb') as handle:
                 pickle.dump(dict_UT_paper_number_authors, handle, protocol = 2)
        print ("written:",'../data/dict_UT_paper_number_authors.pkl', len(dict_UT_paper_number_authors))   





    plos_df = plos_df.sort_values(by=['paper_UT'])
    list_paper_UT = list(plos_df.paper_UT.values)    

    list_number_authors = []
    for paper_UT in list_paper_UT:
        list_number_authors.append(dict_UT_paper_number_authors[paper_UT])

    plos_df['number_authors'] = list_number_authors



    plos_simple = plos_df[['paper_UT','number_authors']]
    df_merged = pd.merge(df_merged, plos_simple, on='paper_UT', how='left')




    # df_merged = df_merged.drop('number_authors_y', axis=1)
    # df_merged = df_merged.rename(columns={'number_authors_x': 'number_authors'})
    # sorted(df_merged.columns)



    path = '../data/df_reference_cite_plos_merged_simplified_added_more_columns.pkl'
    %time df_merged.to_pickle(path, compression='infer', protocol=2)
    print ("written df_reference_cite_plos_merged_simplified_added_more_columns")


    plos_df.head(100)

In [ ]:
ls -rtl ../data/dict*


In [ ]:
# ########  DONE   (no need to re-run)!!!!!!     i get the dict plos paper_UT list of DAIS (disambiguated authors)


list_paper_UT = list(plos_df.paper_UT.values)    



##### i get the list of all DAIS (disambiguated authors) for the list of PLOS papers
try:
    tot_list_DAIS = pickle.load(open('../data/tot_list_DAIS.pkl', 'rb'))    
    dict_plos_paper_UT_list_DAIS = pickle.load(open('../data/dict_plos_paper_UT_list_DAIS.pkl', 'rb'))    
    
except:    

    cont  = 1
    tot_list_DAIS=[]
    dict_plos_paper_UT_list_DAIS = {}
    for paper_UT in list_paper_UT:

        if cont % 10000 == 0:
           print (cont)
        #### i get the list of disambiguated author IDs for the paper plos
        result_query = papers_con.collection.find_one({"UT":paper_UT},{"AU":1})  
         #  print (result_query) ##  example: {'_id': ObjectId('54d3bdcdec29bd464368e4a8'), 'AU': [{'AU': 'Loffler, S', 'DAIS': 19993061}, {'AU': 'Jessen, J', 'DAIS': 37335135}, {'AU': 'Schmid, T', 'DAIS': 30805206}, {'AU': 'Porksen, U', 'DAIS': 63190680}]}



        list_DAIS=[]
        for dict_author in result_query['AU']:  # the result of find_one is a dictionary, NOT an iterator!  And the, result_query['AU'] is a list of dict with info on all the authors of the paper_UT        
            try:
                DAIS = dict_author['DAIS']
                #list_DAIS.append(DAIS)
                tot_list_DAIS.append(DAIS)
                list_DAIS.append(DAIS)
            except :
                pass#print ("author without DAIS",dict_author, "  in paper:", paper_UT)
                #input()
            
            
            
            
        ########## alternative, more pythonic 
        #### example: [d['value'] for d in l if 'value' in d]
        #lista_of_dict =  result_query['AU'] 
        #list_DAIS = [d['DAIS'] for d in lista_of_dict if 'DAIS' in d]
            
                
        dict_plos_paper_UT_list_DAIS[paper_UT] = list_DAIS
        cont += 1


    tot_list_DAIS = list(set(tot_list_DAIS))  # i remove duplicates
    with open('../data/tot_list_DAIS.pkl', 'wb') as handle:
             pickle.dump(tot_list_DAIS, handle, protocol = 2)
    print ("written:",'../data/tot_list_DAIS.pkl')
    
    
    
    
    with open('../data/dict_plos_paper_UT_list_DAIS.pkl', 'wb') as handle:
             pickle.dump(dict_plos_paper_UT_list_DAIS, handle, protocol = 2)
    print ("written:",'../data/dict_plos_paper_UT_list_DAIS.pkl')
                   

print ("# unique DAIS: ",len(tot_list_DAIS), len(dict_plos_paper_UT_list_DAIS))    #  697,993    158,813


    
    
    
    
    
    

    
#### i get dict of DAIS list of all authored papers by that DAIS (disambiguated authors from all PLOS papers) 
try:
    tot_list_papers_authored = pickle.load(open('../data/tot_list_papers_authored.pkl', 'rb'))    
    dict_DAIS_list_papers = pickle.load(open('../data/dict_DAIS_list_papers.pkl', 'rb'))    
except:        
    
    cont  = 1   
    tot_list_papers_authored=[]    
    dict_DAIS_list_papers = {}





    for DAIS in tot_list_DAIS:    # tot:  697993

                  ######for DAIS in aux_list_DAIS:  #  ooooooooooooooojo!!!!!!!!!! this is temporary, remove line after it is done running, and un-comment the previous lines!!!!
       

        if cont % 10000 == 0:
            print (cont)


        #####  i get all the papers by a given disambiguated author
        cursor = dais_con.collection.find({"DAIS":DAIS},{"UT":1}) 

        list_papers=[]
        for item in cursor:  # I iterate over all papers by all the authors of paper_UT
            UT=item["UT"]
            tot_list_papers_authored.append(UT)
            list_papers.append(UT)

        dict_DAIS_list_papers[DAIS] = list_papers

        cont += 1




    tot_list_papers_authored = list(set(tot_list_papers_authored))  # i remove duplicates
#     with open('../data/tot_list_papers_authored.pkl', 'wb') as handle:  # I dont really need this list
#              pickle.dump(tot_list_papers_authored, handle, protocol = 2)
#     print ("written:",'../data/tot_list_papers_authored.pkl')
 


with open('../data/dict_DAIS_list_papers.pkl', 'wb') as handle:
         pickle.dump(dict_DAIS_list_papers, handle, protocol = 2)
print ("written:",'../data/dict_DAIS_list_papers.pkl')
 
         
print ("# unique authored papers by all those DAIS:  ",len(tot_list_papers_authored), len(dict_DAIS_list_papers))









#### now i get the publication year of all papers author by the list of DAIS (from the list of all plos papers)
try:
    dict_all_papers_authored_publ_year = pickle.load(open('../data/dict_all_papers_authored_publ_year.pkl', 'rb'))  
except:
    
    cont  = 1  
    dict_all_papers_authored_publ_year = {}
    for paper in tot_list_papers_authored:
        if cont % 10000 == 0:
            print (cont)
            
        dict_result_query = papers_con.collection.find_one({"UT":paper},{"issue.PY":1})  
        
        try:
            year = dict_result_query['issue']['PY']
            dict_all_papers_authored_publ_year[paper]=year
        except: pass # if the paper does not exist or doesnt have a publication year
            
        cont += 1
                
        
    with open('../data/dict_all_papers_authored_publ_year.pkl', 'wb') as handle:
                 pickle.dump(dict_all_papers_authored_publ_year, handle, protocol = 2)
    print ("written:",'../data/dict_all_papers_authored_publ_year.pkl',len(dict_all_papers_authored_publ_year))   
    
    
    
    
    
    

In [ ]:
df_merged.columns

In [ ]:

########################### next, i get the dictionary of all authored paper UTs vs the list of the R9 references they use  (later i need to convert R9s into UTs)


try:  ####   OJOOOO !  replace by the final dict and list once it is done (instead of the partial)
    #dict_all_authored_paper_UTs_list_R9_references = pickle.load(open('../data/dict_all_authored_paper_UTs_list_R9_references.pkl', 'rb'))  
    #list_all_R9s = pickle.load(open('../data/list_all_R9s.pkl', 'rb'))  
    
    
    
    dict_all_authored_paper_UTs_list_R9_references = pickle.load(open('../data/dict_all_authored_paper_UTs_list_R9_references_partial.pkl', 'rb'))      
    list_all_R9s = pickle.load(open('../data/list_all_R9s_partial.pkl', 'rb'))  
    
    aux_list_authored_papers= []
    for llave in dict_all_papers_authored_publ_year:
        if llave not in dict_all_authored_paper_UTs_list_R9_references:
            aux_list_authored_papers.append(llave)

#except:


    dict_all_authored_paper_UTs_list_R9_references={}   
    list_all_R9s = []

    cont =1
    ###for paper_UT in dict_all_papers_authored_publ_year:     # tot:   12,357,336  with publ year info



    for paper_UT in aux_list_authored_papers:   ##################  ojo!!! this is temporary, remove after done running and un-coment previous lines

        result_query = papers_con.collection.find_one({"UT":paper_UT},{"CR":1})  
        if cont % 10000 == 0:
            print (cont)

        #    result_query['CR']  ### keys of the resulting find_one dictionary:   '_id', 'CR', 'UT'
            ##### result_query['CR'] is a list of dict (one element per reference that the paper UT lists):  
            #  [ {'/A': '*DHHS PAN CLIN PRA', '/W': 'GUID US ANT AG HIV 1', '/Y': '2006'}, 
            #    {'/A': 'BARTLETT, JA',  '/P': '1369',  '/V': '15',  '/W': 'AIDS', '/Y': '2001',  'R9': '0081245001'}]


        try:        
            lista_dict_references =  result_query['CR']  # i access the references used by a given paper_UT

            list_ref_R9 = [d['R9'] for d in lista_dict_references if 'R9' in d]

            list_all_R9s  += list_ref_R9

        except : pass # if the paper doesnt have a list of references




        dict_all_authored_paper_UTs_list_R9_references[paper_UT]=list_ref_R9

        cont +=1
   



with open('../data/dict_all_authored_paper_UTs_list_R9_references.pkl', 'wb') as handle:
             pickle.dump(dict_all_authored_paper_UTs_list_R9_references, handle, protocol = 2)
print ("written:",'../data/dict_all_authored_paper_UTs_list_R9_references.pkl',len(dict_all_authored_paper_UTs_list_R9_references))   





list_all_R9s = list(set(list_all_R9s))

with open('../data/list_all_R9s_partial.pkl', 'wb') as handle:
             pickle.dump(list_all_R9s, handle, protocol = 2)
print ("written:",'../data/list_all_R9s_partial.pkl',len(list_all_R9s))   



In [ ]:
###################  i collect the keys ffrom all the partial dict so far, to continue where i left off:
import pickle

partial_list_keys=[]
i = 1

while i < 11:
    try:
        aux_dict = pickle.load(open('../data/dict_all_authored_paper_UT_list_R9_references_partial'+str(i*1000000)+'.pkl', 'rb'))      
        partial_list_keys += list(aux_dict.keys())
        print (i)
    except: pass
    i += 1

print (len(partial_list_keys))
partial_list_keys = list(set(partial_list_keys))
    
with open('../data/partial_list_keys.pkl', 'wb') as handle:
             pickle.dump(partial_list_keys, handle, protocol = 2)
print ("written:",'../data/partial_list_keys.pkl',len(partial_list_keys))   


In [ ]:
len(partial_list_keys)

In [ ]:
# aux_list_authored_papers= []
#     for llave in dict_all_papers_authored_publ_year:
#         if llave not in dict_all_authored_paper_UTs_list_R9_references:
#             aux_list_authored_papers.append(llave)

In [ ]:
# len(aux_list_authored_papers)   # 11965212
  
# len(dict_all_papers_authored_publ_year)  # 12357336

# len (dict_all_authored_paper_UTs_list_R9_references)  # 392124

#len(list_all_R9s)   11101075


# list_all_R9s = list(set(list_all_R9s))

# print (len(list_all_R9s))    #   6292503



In [ ]:
ls -rtl ../data/dict*


In [ ]:
##########################  FALTA POR CORRER!        transform R9 from list of references of papers into UT

dict_R9_UT = {}
dict_UT_R9 = {}
cont = 1
list_missing_R9s =[]
for R9 in list_all_R9s:
    print (cont)
    
    dict_result = papers_con.collection.find_one({"T9":R9},{"UT":1})    ### keys of the resulting dict_result dictioray:   '_id', 'UT'
    try:
        UT = dict_result['UT']
        dict_R9_UT[R9] = UT
        dict_UT_R9[UT] = R9

    except : 
        list_missing_R9s.append(R9)
    
    if cont % 10000 == 0:
        print (cont)
    cont +=1        
    
    
    
    
with open('../data/dict_R9_UT_partial.pkl', 'wb') as handle:
             pickle.dump(dict_R9_UT, handle, protocol = 2)
print ("written:",'../data/dict_R9_UT_partial.pkl',len(dict_R9_UT))   


    
with open('../data/dict_UT_R9_partial.pkl', 'wb') as handle:
             pickle.dump(dict_UT_R9, handle, protocol = 2)
print ("written:",'../data/dict_UT_R9_partial.pkl',len(dict_UT_R9))   





list_missing_R9s =  set(list(list_missing_R9s))
print (len(list_missing_R9s))

In [ ]:
len(dict_all_authored_paper_UTs_list_R9_references)

In [ ]:
# ##################### FALTA POR CORRER!      i create a new dict of paper_UT : list of ref_UT from the auxiliary dict_all_authored_paper_UTs_list_R9_references:

# dict_all_authored_paper_UT_list_UT_references = {}
# cont = 1
# for paper_UT in dict_all_authored_paper_UTs_list_R9_references:
#     print (cont)
#     list_ref_R9 = dict_all_authored_paper_UTs_list_R9_references[paper_UT]
    
#     list_aux_ref_UT = []
#     for R9 in list_ref_R9:
#         try:
#             UT = dict_R9_UT[R9]
#             list_aux_ref_UT.append(UT)
#         except: pass
        
#     dict_all_authored_paper_UT_list_UT_references[paper_UT] = list_aux_ref_UT
        
#     if cont % 10000 == 0:
#         print (cont)
#     cont +=1  




# with open('../data/dict_all_authored_paper_UT_list_UT_references_partial.pkl', 'wb') as handle:
#              pickle.dump(dict_all_authored_paper_UT_list_UT_references, handle, protocol = 2)
# print ("written:",'../data/dict_all_authored_paper_UT_list_UT_references_partial.pkl',len(dict_all_authored_paper_UT_list_UT_references))   


In [ ]:
plos_df.head()    # 156610
sorted(df_merged.columns) #   5787634
sorted(plos_df.columns) #   5787634

#df_merged.regex_sect_index.value_counts()

In [ ]:
### add new column from list
#plos_df['new_paper_UT_copy'] = lista_plos_UT
plos_df
len(lista_plos_UT)


In [ ]:
####  DONE. the get the number of references used in each section for every plos paper:



plos_df = plos_df.sort_values(by=['paper_UT'])    ### i make sure the order by plos paper UT is fixed, so I add the new columns in their proper order 
lista_plos_UT = list(plos_df.paper_UT.unique())


dict_plos_UT_num_ref_sect0 ={}
dict_plos_UT_num_ref_sect1 ={}
dict_plos_UT_num_ref_sect2 ={}
dict_plos_UT_num_ref_sect3 ={}
dict_plos_UT_num_ref_sect4 ={}
dict_plos_UT_num_ref_sect5 ={}
dict_plos_UT_num_ref_sect6 ={}
dict_plos_UT_num_ref_sect7 ={}




dict_plos_UT_eff_num_ref = {}


for paper_UT in lista_plos_UT:     
    dict_plos_UT_num_ref_sect0[paper_UT] = 0   # i initialize all values to 0, to cover the cases where thre are NO records for a given paper-section
    dict_plos_UT_num_ref_sect1[paper_UT] = 0
    dict_plos_UT_num_ref_sect2[paper_UT] = 0
    dict_plos_UT_num_ref_sect3[paper_UT] = 0
    dict_plos_UT_num_ref_sect4[paper_UT] = 0
    dict_plos_UT_num_ref_sect5[paper_UT] = 0
    dict_plos_UT_num_ref_sect6[paper_UT] = 0
    dict_plos_UT_num_ref_sect7[paper_UT] = 0

    
    

num_records = 0
cont = 1
#for paper_UT in lista_plos_UT:
for paper_UT, group_df in df_merged.groupby(['paper_UT']):  #### OJO!!!! THIS LOOP IS WAY FASTER THAN DOING:  for   paper_UT in list_paper_UT    !!!!    
    print (cont)   
    
   
    dict_plos_UT_eff_num_ref[paper_UT] =len(group_df)
            
    for section in range(8):
        df_selection_by_sect = group_df[ group_df['regex_sect_index'] == section]
                        
        if section ==0:
              dict_plos_UT_num_ref_sect0[paper_UT] = len(df_selection_by_sect)
        elif section ==1:
              dict_plos_UT_num_ref_sect1[paper_UT] = len(df_selection_by_sect)
        elif section ==2:
              dict_plos_UT_num_ref_sect2[paper_UT] = len(df_selection_by_sect)
        elif section ==3:
              dict_plos_UT_num_ref_sect3[paper_UT] = len(df_selection_by_sect)
        elif section ==4:
              dict_plos_UT_num_ref_sect4[paper_UT] = len(df_selection_by_sect)
        elif section ==5:
              dict_plos_UT_num_ref_sect5[paper_UT] = len(df_selection_by_sect)
        elif section ==6:
              dict_plos_UT_num_ref_sect6[paper_UT] = len(df_selection_by_sect)
        elif section ==7:
              dict_plos_UT_num_ref_sect7[paper_UT] = len(df_selection_by_sect)
                                                                              
        
    cont += 1
    num_records  += len(group_df)                                  
                                            
                                            
print ("done creating dictionaries")                                           







plos_df = plos_df.sort_values(by=['paper_UT'])    ### i make sure the order by plos paper UT is fixed, so I add the new columns in their proper order 
lista_plos_UT = list(plos_df.paper_UT.unique())
print (len(lista_plos_UT))

lista_section0 = []
lista_section1 = []
lista_section2 = []
lista_section3 = []
lista_section4 = []
lista_section5 = []
lista_section6 = []
lista_section7 = []

lista_eff_num_papers = []

lista_missing_paper_UT = []
for paper_UT in lista_plos_UT: ### there are 52  paper_UT that have NO references 
    try:
        lista_section0.append(dict_plos_UT_num_ref_sect0[paper_UT])
    except KeyError:   
        lista_section0.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
        
    try:
        lista_section1.append(dict_plos_UT_num_ref_sect1[paper_UT])
    except KeyError:   
        lista_section1.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
            
    try:
        lista_section2.append(dict_plos_UT_num_ref_sect2[paper_UT])
    except KeyError:   
        lista_section2.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
            
    try:
        lista_section3.append(dict_plos_UT_num_ref_sect3[paper_UT])
    except KeyError:   
        lista_section3.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
            
    try:
        lista_section4.append(dict_plos_UT_num_ref_sect4[paper_UT])
    except KeyError:   
        lista_section4.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
            
    try:
        lista_section5.append(dict_plos_UT_num_ref_sect5[paper_UT])
    except KeyError:   
        lista_section5.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
            
    try:
        lista_section6.append(dict_plos_UT_num_ref_sect6[paper_UT])
    except KeyError:   
        lista_section6.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
        
    try:
        lista_section7.append(dict_plos_UT_num_ref_sect7[paper_UT])
    except KeyError:   
        lista_section7.append(np.nan)
        lista_missing_paper_UT.append(paper_UT)
        
        
        
        
    try:    
        lista_eff_num_papers.append(dict_plos_UT_eff_num_ref[paper_UT])
    except KeyError:   
        lista_eff_num_papers.append(np.nan)
        
        
        
    
print (len(plos_df), len(lista_section0), len(lista_section1), len(lista_section2), len(lista_section3), len(lista_section4), len(lista_section5), len(lista_section6), len(lista_section7), len(lista_eff_num_papers))    

plos_df['num_ref_section0'] = lista_section0
plos_df['num_ref_section1'] = lista_section1
plos_df['num_ref_section2'] = lista_section2
plos_df['num_ref_section3'] = lista_section3
plos_df['num_ref_section4'] = lista_section4
plos_df['num_ref_section5'] = lista_section5
plos_df['num_ref_section6'] = lista_section6
plos_df['num_ref_section7'] = lista_section7

plos_df['eff_num_ref'] = lista_eff_num_papers




## example of plos not included:  000207444200009  (does it have 0 references in each section???)   --->>  check df_merged

print ("num missing plos papers:",len(set(lista_missing_paper_UT)))






plos_df['fract_ref_section0'] = plos_df.apply(lambda row: row.num_ref_section0 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section1'] = plos_df.apply(lambda row: row.num_ref_section1 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section2'] = plos_df.apply(lambda row: row.num_ref_section2 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section3'] = plos_df.apply(lambda row: row.num_ref_section3 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section4'] = plos_df.apply(lambda row: row.num_ref_section4 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section5'] = plos_df.apply(lambda row: row.num_ref_section5 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section6'] = plos_df.apply(lambda row: row.num_ref_section6 / row.eff_num_ref, axis=1)
plos_df['fract_ref_section7'] = plos_df.apply(lambda row: row.num_ref_section7 / row.eff_num_ref, axis=1)


plos_df['tot_frac_eff_ref'] = plos_df.apply(lambda row: row.fract_ref_section0 +  row.fract_ref_section1 +  row.fract_ref_section2 +  row.fract_ref_section3 +  row.fract_ref_section4 +  row.fract_ref_section5 + row.fract_ref_section6 +  row.fract_ref_section7 , axis=1)







with open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect.pkl', 'wb') as handle:
                 pickle.dump(plos_df, handle, protocol = 2)
print ("written:",'../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect.pkl',len(plos_df))   





                                            
   
    

In [ ]:
sorted(plos_df.columns)

In [ ]:
####  i get the number of YOUNG AND OLF references used in each section for every plos paper:



time_window_young = 1
time_window_old = 10


# if string_references_age == "young":
#         time_window = 1
#         string_age_selection="only young references from >="+ str((min(years)-time_window_young))
#         preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window_young) ]   
#         print ("  size of preselection3 (only young references):",preselection_df3.shape, string_age_selection)
        
#     elif string_references_age == "old":
#         time_window = 10
#         string_age_selection="only old references from <="+str((min(years)-time_window_old))
#         preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window_old) ]   
#         print ("  size of preselection3 (only young references):",preselection_df3.shape,string_age_selection )



plos_df = plos_df.sort_values(by=['paper_UT'])    ### i make sure the order by plos paper UT is fixed, so I add the new columns in their proper order 
lista_plos_UT = list(plos_df.paper_UT.unique())


dict_plos_UT_num_old_ref_sect0 = {}  # number of old ref per paper and per section
dict_plos_UT_num_old_ref_sect1 = {}
dict_plos_UT_num_old_ref_sect2 = {}
dict_plos_UT_num_old_ref_sect3 = {}
dict_plos_UT_num_old_ref_sect4 = {}
dict_plos_UT_num_old_ref_sect5 = {}
dict_plos_UT_num_old_ref_sect6 = {}
dict_plos_UT_num_old_ref_sect7 = {}



dict_plos_UT_num_young_ref_sect0 = {}  # number of old ref per paper and per section
dict_plos_UT_num_young_ref_sect1 = {}
dict_plos_UT_num_young_ref_sect2 = {}
dict_plos_UT_num_young_ref_sect3 = {}
dict_plos_UT_num_young_ref_sect4 = {}
dict_plos_UT_num_young_ref_sect5 = {}
dict_plos_UT_num_young_ref_sect6 = {}
dict_plos_UT_num_young_ref_sect7 = {}




for paper_UT in lista_plos_UT:                 
    
    dict_plos_UT_num_old_ref_sect0[paper_UT] = 0   # i initialize all values to 0, to cover the cases where thre are NO records for a given paper-section
    dict_plos_UT_num_old_ref_sect1[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect2[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect3[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect4[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect5[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect6[paper_UT] = 0
    dict_plos_UT_num_old_ref_sect7[paper_UT] = 0

    
          
    dict_plos_UT_num_young_ref_sect0[paper_UT] = 0   # i initialize all values to 0, to cover the cases where thre are NO records for a given paper-section
    dict_plos_UT_num_young_ref_sect1[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect2[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect3[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect4[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect5[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect6[paper_UT] = 0
    dict_plos_UT_num_young_ref_sect7[paper_UT] = 0  
    


    
    

    
    
    
cont = 1
#for paper_UT in lista_plos_UT:
for paper_UT, group_df in df_merged.groupby(['paper_UT']):  #### OJO!!!! THIS LOOP IS WAY FASTER THAN DOING:  for   paper_UT in list_paper_UT    !!!!    
    print (cont)   
    
    
    year = int(group_df.plos_pub_year.unique())
    
    preselection_df_young = group_df[group_df['ref_pub_year'] >= (year-time_window_young) ]   
    #print ("  size of preselection (only young references):", group_df.shape, preselection_df_young.shape)
 

    preselection_df_old = group_df[group_df['ref_pub_year'] <= (year-time_window_old) ]   
   # print ("  size of preselection (only young references):", group_df.shape, preselection_df_old.shape)
 
    
        
            
    for section in range(8):
        df_selection_young_by_sect = preselection_df_young[ preselection_df_young['regex_sect_index'] == section]
        df_selection_old_by_sect = preselection_df_old[ preselection_df_old['regex_sect_index'] == section]
       
    
    
        if section ==0:
            dict_plos_UT_num_young_ref_sect0[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect0[paper_UT] = len(df_selection_old_by_sect)
        elif section ==1:
            dict_plos_UT_num_young_ref_sect1[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect1[paper_UT] = len(df_selection_old_by_sect)
        elif section ==2:
            dict_plos_UT_num_young_ref_sect2[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect2[paper_UT] = len(df_selection_old_by_sect)
        elif section ==3:
            dict_plos_UT_num_young_ref_sect3[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect3[paper_UT] = len(df_selection_old_by_sect)
        elif section ==4:
            dict_plos_UT_num_young_ref_sect4[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect4[paper_UT] = len(df_selection_old_by_sect)
        elif section ==5:
            dict_plos_UT_num_young_ref_sect5[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect5[paper_UT] = len(df_selection_old_by_sect)
        elif section ==6:
            dict_plos_UT_num_young_ref_sect6[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect6[paper_UT] = len(df_selection_old_by_sect)
        elif section ==7:
            dict_plos_UT_num_young_ref_sect7[paper_UT] = len(df_selection_young_by_sect)
            dict_plos_UT_num_old_ref_sect7[paper_UT] = len(df_selection_old_by_sect)
                                                                              
        
        
        
        
    cont += 1
                          
                                            
                                            
print ("done creating dictionaries")                                           










plos_df = plos_df.sort_values(by=['paper_UT'])    ### i make sure the order by plos paper UT is fixed, so I add the new columns in their proper order 
lista_plos_UT = list(plos_df.paper_UT.unique())
print (len(lista_plos_UT))

lista_young_section0 = []
lista_young_section1 = []
lista_young_section2 = []
lista_young_section3 = []
lista_young_section4 = []
lista_young_section5 = []
lista_young_section6 = []
lista_young_section7 = []



lista_old_section0 = []
lista_old_section1 = []
lista_old_section2 = []
lista_old_section3 = []
lista_old_section4 = []
lista_old_section5 = []
lista_old_section6 = []
lista_old_section7 = []




lista_missing_paper_UT = []
for paper_UT in lista_plos_UT: ### there are 52  paper_UT that have NO references 
    
    
    #### for young ref
    try:
        lista_young_section0.append(dict_plos_UT_num_young_ref_sect0[paper_UT])
    except KeyError:   
        lista_young_section0.append(np.nan)
        
        
    try:
        lista_young_section1.append(dict_plos_UT_num_young_ref_sect1[paper_UT])
    except KeyError:   
        lista_young_section1.append(np.nan)
       
            
    try:
        lista_young_section2.append(dict_plos_UT_num_young_ref_sect2[paper_UT])
    except KeyError:   
        lista_young_section2.append(np.nan)
       
            
    try:
        lista_young_section3.append(dict_plos_UT_num_young_ref_sect3[paper_UT])
    except KeyError:   
        lista_young_section3.append(np.nan)
        
            
    try:
        lista_young_section4.append(dict_plos_UT_num_young_ref_sect4[paper_UT])
    except KeyError:   
        lista_young_section4.append(np.nan)
        
            
    try:
        lista_young_section5.append(dict_plos_UT_num_young_ref_sect5[paper_UT])
    except KeyError:   
        lista_young_section5.append(np.nan)
       
            
    try:
        lista_young_section6.append(dict_plos_UT_num_young_ref_sect6[paper_UT])
    except KeyError:   
        lista_young_section6.append(np.nan)
        
        
    try:
        lista_young_section7.append(dict_plos_UT_num_young_ref_sect7[paper_UT])
    except KeyError:   
        lista_young_section7.append(np.nan)
        
     
    
    
    
    ###### for old ref
    try:
        lista_old_section0.append(dict_plos_UT_num_old_ref_sect0[paper_UT])
    except KeyError:   
        lista_old_section0.append(np.nan)
        
        
    try:
        lista_old_section1.append(dict_plos_UT_num_old_ref_sect1[paper_UT])
    except KeyError:   
        lista_old_section1.append(np.nan)
        
            
    try:
        lista_old_section2.append(dict_plos_UT_num_old_ref_sect2[paper_UT])
    except KeyError:   
        lista_old_section2.append(np.nan)
       
            
    try:
        lista_old_section3.append(dict_plos_UT_num_old_ref_sect3[paper_UT])
    except KeyError:   
        lista_old_section3.append(np.nan)
       
            
    try:
        lista_old_section4.append(dict_plos_UT_num_old_ref_sect4[paper_UT])
    except KeyError:   
        lista_old_section4.append(np.nan)
        
            
    try:
        lista_old_section5.append(dict_plos_UT_num_old_ref_sect5[paper_UT])
    except KeyError:   
        lista_old_section5.append(np.nan)
       
            
    try:
        lista_old_section6.append(dict_plos_UT_num_old_ref_sect6[paper_UT])
    except KeyError:   
        lista_old_section6.append(np.nan)
        
        
    try:
        lista_old_section7.append(dict_plos_UT_num_old_ref_sect7[paper_UT])
    except KeyError:   
        lista_old_section7.append(np.nan)
        
        
     
        
        
print (len(plos_df), len(lista_young_section0), len(lista_young_section1), len(lista_young_section2), len(lista_young_section3), len(lista_young_section4), len(lista_young_section5), len(lista_young_section6), len(lista_young_section7)   ) 
print (len(plos_df), len(lista_old_section0), len(lista_old_section1), len(lista_old_section2), len(lista_old_section3), len(lista_old_section4), len(lista_old_section5), len(lista_old_section6), len(lista_old_section7)   ) 



plos_df['num_young_ref_section0'] = lista_young_section0
plos_df['num_young_ref_section1'] = lista_young_section1
plos_df['num_young_ref_section2'] = lista_young_section2
plos_df['num_young_ref_section3'] = lista_young_section3
plos_df['num_young_ref_section4'] = lista_young_section4
plos_df['num_young_ref_section5'] = lista_young_section5
plos_df['num_young_ref_section6'] = lista_young_section6
plos_df['num_young_ref_section7'] = lista_young_section7

       

plos_df['num_old_ref_section0'] = lista_old_section0
plos_df['num_old_ref_section1'] = lista_old_section1
plos_df['num_old_ref_section2'] = lista_old_section2
plos_df['num_old_ref_section3'] = lista_old_section3
plos_df['num_old_ref_section4'] = lista_old_section4
plos_df['num_old_ref_section5'] = lista_old_section5
plos_df['num_old_ref_section6'] = lista_old_section6
plos_df['num_old_ref_section7'] = lista_old_section7






plos_df['fract_young_ref_section0'] = plos_df.apply(lambda row: row.num_young_ref_section0 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section1'] = plos_df.apply(lambda row: row.num_young_ref_section1 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section2'] = plos_df.apply(lambda row: row.num_young_ref_section2 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section3'] = plos_df.apply(lambda row: row.num_young_ref_section3 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section4'] = plos_df.apply(lambda row: row.num_young_ref_section4 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section5'] = plos_df.apply(lambda row: row.num_young_ref_section5 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section6'] = plos_df.apply(lambda row: row.num_young_ref_section6 / row.eff_num_ref, axis=1)
plos_df['fract_young_ref_section7'] = plos_df.apply(lambda row: row.num_young_ref_section7 / row.eff_num_ref, axis=1)






plos_df['fract_old_ref_section0'] = plos_df.apply(lambda row: row.num_old_ref_section0 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section1'] = plos_df.apply(lambda row: row.num_old_ref_section1 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section2'] = plos_df.apply(lambda row: row.num_old_ref_section2 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section3'] = plos_df.apply(lambda row: row.num_old_ref_section3 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section4'] = plos_df.apply(lambda row: row.num_old_ref_section4 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section5'] = plos_df.apply(lambda row: row.num_old_ref_section5 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section6'] = plos_df.apply(lambda row: row.num_old_ref_section6 / row.eff_num_ref, axis=1)
plos_df['fract_old_ref_section7'] = plos_df.apply(lambda row: row.num_old_ref_section7 / row.eff_num_ref, axis=1)




plos_df['tot_frac_eff_young_ref'] = plos_df.apply(lambda row: row.fract_young_ref_section0 +  row.fract_young_ref_section1 +  row.fract_young_ref_section2 +  row.fract_young_ref_section3 +  row.fract_young_ref_section4 +  row.fract_young_ref_section5 + row.fract_young_ref_section6 +  row.fract_young_ref_section7 , axis=1)
plos_df['tot_frac_eff_old_ref'] = plos_df.apply(lambda row:   row.fract_old_ref_section0 +    row.fract_old_ref_section1 +    row.fract_old_ref_section2 +  row.fract_old_ref_section3 +  row.fract_old_ref_section4 +  row.fract_old_ref_section5 + row.fract_old_ref_section6 +  row.fract_old_ref_section7 , axis=1)


plos_df['tot_num_eff_young_ref'] = plos_df.apply(lambda row: row.num_young_ref_section0 +  row.num_young_ref_section1 +  row.num_young_ref_section2 +  row.num_young_ref_section3 +  row.num_young_ref_section4 +  row.num_young_ref_section5 + row.num_young_ref_section6 +  row.num_young_ref_section7 , axis=1)
plos_df['tot_num_eff_old_ref'] = plos_df.apply(lambda row:   row.num_old_ref_section0 +    row.num_old_ref_section1 +    row.num_old_ref_section2 +  row.num_old_ref_section3 +  row.num_old_ref_section4 +  row.num_old_ref_section5 + row.num_old_ref_section6 +  row.num_old_ref_section7 , axis=1)











with open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'wb') as handle:
                 pickle.dump(plos_df, handle, protocol = 2)
print ("written:",'../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl',len(plos_df))   







    
    

In [ ]:
sorted(plos_df.columns)
print (plos_df.shape)

plos_df[['paper_UT','total_refs','eff_num_ref','tot_num_eff_young_ref','tot_num_eff_old_ref','tot_frac_eff_young_ref','tot_frac_eff_old_ref','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7', 'num_young_ref_section0','num_young_ref_section1','num_young_ref_section2','num_young_ref_section3',  'num_old_ref_section0','num_old_ref_section1','num_old_ref_section2','num_old_ref_section3']]




sorted(plos_df.columns)
print (plos_df.shape)

plos_df[['paper_UT','total_refs','eff_num_ref','tot_num_eff_young_ref','tot_num_eff_old_ref','fract_young_ref_section0','fract_young_ref_section1',  'fract_old_ref_section0','fract_old_ref_section1']]

In [ ]:
sorted(plos_df.columns)

In [ ]:
#   double check:
print (df_merged.shape)   # (5787634, 27)
df_merged=df_merged.drop_duplicates(subset=['paper_UT','reference_UT','regex_sect_index'])
print (df_merged.shape)  # (5787634, 27)       ### yay!!!! df_merged EXCLUDES repetitions of a given reference in a given section of a plos paper

In [ ]:
plos_df.hist(column='eff_num_ref', bins=100)

In [ ]:
# df_select = df_merged[df_merged['paper_UT'] == '000207443600001']
# print (len(df_select))
# df_select[['paper_UT','reference_UT','regex_sect_index','occurence']]

In [ ]:
plos_df.hist(column='total_refs', bins=100)

In [ ]:
#print (lista_plos_UT[:10])
#df_merged[df_merged['paper_UT'].isin(lista_plos_UT[:10])]

# '000207444200009',
#  '000207455200002',
#  '000267347900002',
#  '000268637600014',
#  '000270594000017',
#  '000278017400008',
#  '000280065600013',
#  '000284686500004',
#  '000286663900015',
#  '000287764100045',
#  '000289058700017',
#  '000291097600002',
#  '000306507000030',
#  '000309510900003',
#  '000311997100014',
#  '000313236200097',
#  '000313236200126',
#  '000315157200061',
#  '000315159200027',
#  '000315637900010',
#  '000315897100001',
#  '000320579400001',
#  '000322167900019',
#  '000323221500028',
#  '000324238400040',
#  '000324527300053'

set(lista_missing_paper_UT)

plos_df[plos_df['paper_UT'] == '000207444200009']

In [ ]:
# len(df_merged.reference_UT.unique())
# df_merged.shape
# len(set(lista_plos_UT)), len(plos_df), len(dict_plos_UT_num_ref_sect0)


In [ ]:
len(dict_plos_UT_num_ref_sect0), len(dict_plos_UT_num_ref_sect1),len(dict_plos_UT_num_ref_sect2) ,len(dict_plos_UT_num_ref_sect3) 
#plos_df

In [ ]:
len(df_merged.groupby(['paper_UT']))

In [ ]:
#len(dict_DAIS_list_papers)   #  13,052,537

# len(dict_all_papers_authored_publ_year)  # 12,357,336

In [ ]:
### adding columns regarding number of citations as of 2009, 2011, 2013 of only young references!


                                            
%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)






In [ ]:
sorted(df_merged.columns)

# ['categ_codes',
#  'cite_count',
#  'diff_year_plos_ref',
#  'isolated_citation',
#  'log2_num_cit_paper',
#  'log2_num_cit_ref',
#  'number_authors',
#  'occurence',
#  'paper_UT',
#  'paper_cite_count',
#  'plos_article_type',
#  'plos_field',
#  'plos_j1',
#  'plos_pub_year',
#  'ref_field',
#  'ref_j1',
#  'ref_pub_year',
#  'reference_UT',
#  'reference_rank',
#  'regex_sect_index',
#  'rel_loc_in_sect',
#  'sect_char_pos',
#  'sect_char_total',
#  'self_citation',
#  'total_refs']




#df_merged.plos_pub_year.max()  2005-2016

#df_merged.ref_pub_year.min() #  1900-2016

#df_merged.shape   # (5787634, 25)

len(df_merged.paper_UT.unique())    #  156558


len(df_merged.reference_UT.unique())    #  2320777

list_all_ref_UT = df_merged.reference_UT.unique()   # this is an array


list_all_ref_UT

In [ ]:
# ######## extract number of citations of young references early on.


### NON-EFICIENT QUERIES

# focus_plos_year = 2011  # the main figures of my paper correspond to plos papers published in 2011, and the young references are at most one year old == 2011: 
# ############# how many citations did young references have by 2011 (by focus year)???




# df_selection_general_young_ref = df_merged[ (df_merged['ref_pub_year'] >= (focus_plos_year - 1) )]  


# list_UT_young_ref_by_focus_year = df_selection_general_young_ref.reference_UT.unique()






# print ("Total # records of young ref. by", focus_year, ": ",df_selection_general_young_ref.shape, "  # unique ref_UTs:", len(list_UT_young_ref_by_focus_year))   # 


# dict_UT_young_ref_num_cit_by_focus_year = {}

# list_all_citing_appers_of_ref = []   


# for ref_UT in list_all_ref_UT:  # i need to assign a value to all references  (those that were older, get a NaN, those that are young enough (one year old or younger), get whatever number of citations they had by focus_year)           
#     dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = np.nan  # first i initialize all ref to nan  (faster than evaluating whether the reference is in the list of selected ones or not)
    
    
    
    
    
# cont = 0    
# for ref_UT in list_UT_young_ref_by_focus_year:    
#     ref_UT = str(ref_UT)
   

   

# #     cursor = papers_con.collection.find({"UT" : ref_UT}, {"UT":1,"issue.PY":1,'citations':1})  # THIS IS AN ITERATOR
#     item = papers_con.collection.find_one({"UT" : ref_UT}, {"UT":1,"issue.PY":1,'citations':1})  # THIS IS AN ITERATOR
#     # citations:   cites received by the paper (currently updated value!)


#     ## iteration
# #     for item in cursor:  # query (cursor) is an iterator (once i iterate over it once, it is empty), and every item is a dict.    ONLY one element in this particular cursor!!
        
#     #focus_ref = str(item["UT"])
#     year_ref =  item['issue']['PY']



#     dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = 0

#     try:
#         list_cit = item["citations"]                                
#     except KeyError:
#         list_cit=[]

#     aux_list_citing_papers = list_cit

#     cursor2 = papers_con.collection.find( { "UT" :{"$in":aux_list_citing_papers}}, {"UT":1,"issue.PY":1,'citations':1})

    

#     for item2 in cursor2:  # query (cursor) is an iterator (once i iterate over it once, it is empty), and every item is a dict        


#         UT_citing=str(item2["UT"])
#         year = item2['issue']['PY']
#         #print (year)

#         if year <= focus_plos_year:  # is the focus reference has recived the reference at most during the focus year, then i count it
#             dict_UT_young_ref_num_cit_by_focus_year[ref_UT] +=1

       






        
#     print(cont, ref_UT, "# cit total:", len(list_cit),"   # cit by",focus_year, ":",dict_UT_young_ref_num_cit_by_focus_year[ref_UT])        
#    # input()
        
        
        
        
   
        
#     cont +=1  
        
# #   example of all citations received by '000286133900010':     
  
# # [  "000312433100012", "000291741700004", "000293533000004", "000293533000008", "000293955200010", "000318967500010", "000321851900003", "000321851900005", "000321851900006","000321851900007", "000324877000001", "000302254500012", "000326340100029", "000329043200014", "000330353000004", "000330641700001", "000317239300013", "000299059700011", "000302154300005",  "000291741700003", "000297708200027", "000317239300002", "000297708200012", "000343049600116", "000347095400001", "000352084800063", "000353713900026", "000354607300001", "000401743700010"]





 

# print ("done.", len(dict_UT_young_ref_num_cit_by_focus_year),"dumping dict ref.......")         
# with open('../data/dict_UT_young_ref_in'+str(focus_year)+'_num_cit_by_'+str(focus_year)+'.pkl', 'wb') as handle:
#          pickle.dump(dict_UT_young_ref_num_cit_by_focus_year, handle, protocol = 2)
# print ("written:",'../data/dict_UT_young_ref_in'+str(focus_year)+'_num_cit_by_'+str(focus_year)+'.pkl')   




# dict_UT_young_ref_num_cit_by_focus_year


  


In [ ]:

len(dict_UT_young_ref_num_cit_by_focus_year)

aux_dict = dict_UT_young_ref_num_cit_by_focus_year

In [ ]:


# ######## extract number of citations of young references early on.


# focus_plos_year = 2011  # the main figures of my paper correspond to plos papers published in 2011, and the young references are at most one year old == 2011: 
# ############# how many citations did young references have by 2011 (by focus year)???




# df_selection_general_young_ref = df_merged[ (df_merged['ref_pub_year'] >= (focus_plos_year - 1) )]  


# list_UT_young_ref_by_focus_year = df_selection_general_young_ref.reference_UT.unique()






# print ("Total # records of young ref. by", focus_year, ": ",df_selection_general_young_ref.shape, "  # unique ref_UTs:", len(list_UT_young_ref_by_focus_year))   # 



# dict_UT_young_ref_num_cit_by_focus_year = {}

# list_all_citing_appers_of_ref = []   


# for ref_UT in list_all_ref_UT:  # i need to assign a value to all references  (those that were older, get a NaN, those that are young enough (one year old or younger), get whatever number of citations they had by focus_year)           
#     dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = np.nan  # first i initialize all ref to nan  (faster than evaluating whether the reference is in the list of selected ones or not)
    
    
    
    
    
# cont = 0      # 190644#
# #for ref_UT in list_aux_UT:
# for ref_UT in list_UT_young_ref_by_focus_year:  
    
    
    
    
#     ref_UT = str(ref_UT)  # i make sure it is a string
#    # print ("\n",cont, ref_UT)       
    

# #    item = papers_con.collection.find_one({"UT" : ref_UT}, {"UT":1,"issue.PY":1,'citations':1})  # the output of a find_one cursor is a DICTIONARY!!!

#     item = papers_con.collection.find_one({"UT" : ref_UT}, {'citations':1})  # the output of a find_one cursor is a DICTIONARY!!!

#     # citations:   cites received by the paper (currently updated value!)
        

#     dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = 0

#     try:
#         list_citing_papers = item["citations"]                                
    
#         aux_list_citing_papers = list_citing_papers



#         ### QUERY TO GET ALL CITATIONS RECEIVED BY THE REFERENCE   (i dont really need it)
#        # count_cursor1 = papers_con.collection.find( { "UT" :{"$in":aux_list_citing_papers} },    {"UT":1,"issue.PY":1,'citations':1}).count()  ## selects the documents where the value of the field is less than or equal to (i.e. <=) the specified value.
#         #print(cont, ref_UT, "# tot cit:",count_cursor1)




#         ### QUERY TO GET ONLY CITATIONS RECEIVED BY THE REFERENCE AROUND THE FOCUS_YEAR
#         #cursor2_count = papers_con.collection.find( {  "UT" :{"$in":aux_list_citing_papers }, "issue.PY": { '$lte': focus_plos_year }}   ,    {"UT":1,"issue.PY":1,'citations':1}).count()
#         cursor2_count = papers_con.collection.find( {  "UT" :{"$in":aux_list_citing_papers }, "issue.PY": { '$lte': focus_plos_year }} ).count()  #i only care about how many fulfill the conditions

#         dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = cursor2_count

        
        
#     except :  # IF NO CITATIONS IN WOS RECORDS FOR THAT PAPER
# #         list_citing_papers=[]
#          dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = 0
        



#     print(cont)#, ref_UT, "# tot cit:",count_cursor1, "# cit by",focus_year, ":",dict_UT_young_ref_num_cit_by_focus_year[ref_UT])       



 
#    # input()
        
        
        
        
   
        
#     cont +=1  
        
        
  






 

# print ("done.", len(dict_UT_young_ref_num_cit_by_focus_year),"dumping dict ref.......")         
# with open('../data/dict_UT_young_ref_in'+str(focus_year)+'_num_cit_by_'+str(focus_year)+'.pkl', 'wb') as handle:
#          pickle.dump(dict_UT_young_ref_num_cit_by_focus_year, handle, protocol = 2)
# print ("written:",'../data/dict_UT_young_ref_in'+str(focus_year)+'_num_cit_by_'+str(focus_year)+'.pkl')   




# # dict_UT_young_ref_num_cit_by_focus_year


  


In [ ]:

# list_aux_UT= []

# for UT in list_UT_young_ref_by_focus_year:
#     value= dict_UT_young_ref_num_cit_by_focus_year[UT]
#     if value >=0:    # OJO! THIS IS A COMPARISON WITH NP.NAN! (WHICH IS NEVER TRUE, EITHER FOR <= NOR FOR >=)
#         pass
#     else:
#         list_aux_UT.append(UT)

        
# print (len(list_aux_UT), len(dict_UT_young_ref_num_cit_by_focus_year), len(list_UT_young_ref_by_focus_year))
# # n = np.nan

# # if n >= 0:
# #     print (1)

# # 190644 + 438100


len(dict_UT_young_ref_num_cit_by_focus_year)

In [ ]:
#############  REMOVE PREVIOUS VERSION (CELL WITHOUT LOOP OVER FOCUS_YEARS) AFTER RUNNING (THIS ONE IS A GENERAL VERSION WITH A LOOK OVER FOCUS_YEARS)

######## extract number of citations of young references early on.


list_focus_years = [2009,2010,2011,2012,2013]


def_young = 1  # young references are <= 1 year old



#focus_plos_year = 2011  # the main figures of my paper correspond to plos papers published in 2011, and the young references are at most one year old == 2011: 
############# how many citations did young references have by 2011 (by focus year)???


for focus_plos_year in list_focus_years:
    
    

    df_selection_general_young_ref = df_merged[ (df_merged['ref_pub_year'] >= (focus_plos_year - def_young) )]  


    list_UT_young_ref_by_focus_year = df_selection_general_young_ref.reference_UT.unique()






    print ("\n\nTotal # records of young ref. by", focus_plos_year, ": ",df_selection_general_young_ref.shape, "  # unique ref_UTs:", len(list_UT_young_ref_by_focus_year))   


    dict_UT_young_ref_num_cit_by_focus_year = {}

    list_all_citing_appers_of_ref = []   


    for ref_UT in list_all_ref_UT:  # i need to assign a value to all references  (those that were older, get a NaN, those that are young enough (one year old or younger), get whatever number of citations they had by focus_year)           
        dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = np.nan  # first i initialize all ref to nan  (faster than evaluating whether the reference is in the list of selected ones or not)




    cont = 0    
    for ref_UT in list_UT_young_ref_by_focus_year:  




        ref_UT = str(ref_UT)  # i make sure it is a string
       # print ("\n",cont, ref_UT)       


        item = papers_con.collection.find_one({"UT" : ref_UT}, {'citations':1})  # the output of a find_one cursor is a DICTIONARY!!!   citations:   cites received by the paper (currently updated value!)

     
        dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = 0

        try:
            list_citing_papers = item["citations"]                                

            aux_list_citing_papers = list_citing_papers



            ### QUERY TO GET ALL CITATIONS RECEIVED BY THE REFERENCE   (i dont really need it)
           # count_cursor1 = papers_con.collection.find( { "UT" :{"$in":aux_list_citing_papers} },    {"UT":1,"issue.PY":1,'citations':1}).count()  ## selects the documents where the value of the field is less than or equal to (i.e. <=) the specified value.
            #print(cont, ref_UT, "# tot cit:",count_cursor1)




            ### QUERY TO GET ONLY CITATIONS RECEIVED BY THE REFERENCE AROUND THE FOCUS_YEAR
            #cursor2_count = papers_con.collection.find( {  "UT" :{"$in":aux_list_citing_papers }, "issue.PY": { '$lte': focus_plos_year }}   ,    {"UT":1,"issue.PY":1,'citations':1}).count()
            cursor2_count = papers_con.collection.find( {  "UT" :{"$in":aux_list_citing_papers }, "issue.PY": { '$lte': focus_plos_year }} ).count()  #i only care about how many fulfill the conditions

            #        ### MONGODB:   find with multiple conditions: find( {condition1, condition2}, {retrieve_field1, retrieve_field2})   # NO EXTRA BRACKETS, OR IT WONT WORK!!!!

            
            dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = cursor2_count



        except :  # IF NO CITATIONS IN WOS RECORDS FOR THAT PAPER   
             dict_UT_young_ref_num_cit_by_focus_year[ref_UT] = 0




        print(cont)#, ref_UT, "# tot cit:",count_cursor1, "# cit by",focus_year, ":",dict_UT_young_ref_num_cit_by_focus_year[ref_UT])       




        cont +=1  







    print ("done.", len(dict_UT_young_ref_num_cit_by_focus_year),"dumping dict ref.......")         
    with open('../data/dict_UT_young_ref_in'+str(focus_plos_year)+'_num_cit_by_'+str(focus_plos_year)+'.pkl', 'wb') as handle:
             pickle.dump(dict_UT_young_ref_num_cit_by_focus_year, handle, protocol = 2)
    print ("written:",'../data/dict_UT_young_ref_in'+str(focus_plos_year)+'_num_cit_by_'+str(focus_plos_year)+'.pkl')   




  


In [ ]:
# 000290014500020 # tot cit: 22 # cit by 2011 : 1

# 000274470200014 # tot cit: 15 # cit by 2011 : 2


In [ ]:
# preselection_df = df_merged[df_merged['reference_UT'].isin(list_ref_UT_post2005)] 

# preselection_df
dict_UT_young_ref_num_cit_by_focus_year

In [ ]:
# df_merged[df_merged['self_citation']==1][['paper_UT', 'reference_UT']]
print (len(set(tot_list_papers)))


In [ ]:


# lista_all_plos_UTs = pickle.load(open('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/lista_all_plos_UTs.pkl', 'rb'))
# new_lista_all_plos_UTs = ['000'+str(item) for item in lista_all_plos_UTs]   # i need to add the 000 so it matches the db UTs

# #print (len(new_lista_all_plos_UTs))


# # In[15]:




# #lista_UTs=['000322590800016','000254928800015','000305349100026','000321341000076']
# dict_UT_plos_DOI={}
# dict_DOI_plos_UT={}


# start=0
# stop=1000
# while stop <=159000:


#     lista=new_lista_all_plos_UTs[start:stop]
#     query = papers_con.collection.find({"UT":{"$in":lista}},{"UT":1,"AR":1}, no_cursor_timeout=True)  


#     #print (start, stop)

#     for item in query:  # query (cursor) is an iterator (once i iterate over it once, it is empty), and every item is a dict

#         UT=item["UT"]
#         doi=item['AR'][-1]
        
#         #string =str(UT)+" "+str(doi)
#         if ".pone." in doi:
#             #print ( doi)
#             dict_UT_plos_DOI[UT]=doi
            
#             dict_DOI_plos_UT[doi]=UT

#     stop +=1000
#     start +=1000
        
     
#     query.close()  # because i am using the no_cursor_timeout=True, i need also this, or cursor keeps waiting so ur resources are used up
        
# print ("done.", len(dict_UT_plos_DOI))   # 142723 from plos ONE

# # DOI 10.1371/journal.pone.0004056
# # DOI 10.1371/journal.pone.0004126
# # DOI 10.1371/journal.pone.0004392
# # DOI 10.1371/journal.pone.0004404
# # DOI 10.1371/journal.pone.0004432
# # DOI 10.1371/journal.pone.0004463

In [ ]:

# dict_UT_plos_subject={}
# lista_UT=[]
# lista_subjects=[]

# with open('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/plos_subjects/plos_subjects_N100.json') as json_data:
#     d = json.load(json_data) # a dict where the key is the doi and the value is another dict
#     for doi in d:
#         subject=str(d[doi]['subject_level_1']).replace("[","").replace("]","").replace(", ","-").replace("'","")
       
#         if ".pone." in doi:
#             #print (doi)
#             try:
#                 UT = dict_DOI_plos_UT['DOI '+doi]
#                 dict_UT_plos_subject[UT]=subject
#                 lista_UT.append(UT)
#                 lista_subjects.append(subject)
#             #input()
#                 print (UT,doi, subject)
#             except KeyError: pass
#     #df[df['A'].str.contains("Hello|Britain")==True]     or    df[df['model'].str.contains('ac')]
    
# print ("done.", len(dict_UT_plos_subject))

In [ ]:
for item in sorted(dict_UT_plos_DOI.items(), key=operator.itemgetter(1)):
    print (item)

In [ ]:

# lista_llaves=list(dict_UT_plos_subject.keys())
# lista_values=list(dict_UT_plos_subject.values())
# new_df =  pd.DataFrame(
#     {'paper_UT': lista_llaves,
#      'subject': lista_values
#     })

# new_df.head()

In [ ]:
# new_df1 =  pd.DataFrame(
#     {'paper_UT': lista_UT,
#      'subject': lista_subjects
#     })

# new_df1.subject.unique()

In [ ]:
# field_string =  'Physic'
# new_df1[new_df1['subject'].str.contains(field_string)]

In [ ]:
# lista_peq_UTs=['000351275700019','000351275700002','000351275700021','000351275700005']